In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime 
import time

import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [3]:
tf.config.experimental.list_physical_devices(device_type = None)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU')]

In [4]:
f16 = pd.read_csv('data/f16_l3.csv')

In [5]:
X = f16['Force']
y = f16['Acceleration3']

In [6]:
#X = MinMaxScaler().fit_transform(X.values.reshape(-1,1))

In [7]:
#Declare the number of model iterations we will run 
test_iterations = 250

#Declare the size of the initial train test split
test_size = 0.20

#Declare the number of hidden layers to add
layers = 2

#Declare the density of the hidden layers
density = 64

#Declare the type of activation for the hidden layers
activation = 'relu' 
#activation = ks.layers.LeakyReLU(alpha = 0.01)

#Assign the loss function the model will use to train
loss = 'mean_squared_error'

#Assign the optimizer for the compilitation
optimizer = 'adam'

#Declare the patience count used for early stopping
patience = 25

#Declare the test size used for early stopping train test split
es_test_size = 0.25

#Declare the batch size for use in the model
batch_size = 64

#Declare the maximum number of epochs for our model
epochs = 250

In [8]:
X_train, X_true, y_train, y_true = train_test_split(
    X, y, random_state = 8669, test_size = test_size)

In [9]:
#Assigns the computation to be performed via GPU Device:0
with tf.device('/gpu:1'):
    model = Sequential()

    #Creates a for loop that will add the number of layers based on the variable declared in the previous cell    
    for i in range(layers):
        #Adds a model layer with density and activation based on the variables declared in the previous cell
        model.add(Dense(density, activation = activation))
    
    model.add(Dense(1, activation='linear'))
    
    #model.compile(loss = loss, optimizer = optimizer, metrics = ['mse'])

In [10]:
es = EarlyStopping(monitor = 'loss', patience = patience, restore_best_weights = True)
mc = ModelCheckpoint(filepath = 'test_model.h5', monitor = 'loss', save_best_only=True)
X_es_train, X_es_true, y_es_train, y_es_true = train_test_split(X_train, y_train, test_size = es_test_size, random_state = 8669)

In [11]:
#Creates a dataframe by which we will eventually put in our list created above
model_record = pd.DataFrame(columns = ['model_num', 'loss_type', 'time', 'epochs', 'r2', 'mae', 'mse', 'rmse', 'max_error'])

#Creates a dataframe by which our model's predicted values and true values will be stored
predict_record = pd.DataFrame(y_true).reset_index(drop = True)

#Creates a numpy array by which the for loop will use to count model runs and is then used to name df columns
model_counter = np.array([0])

In [12]:
#Assigns the computation to be performed via GPU Device:0
with tf.device('/gpu:1'):
    
#Performs model training repeatedly based on the variable declared previously    
    for i in range(test_iterations):
        model.compile(loss = loss, optimizer = optimizer, metrics = ['mse'])

        #Creates an empty list for storing model metrics and other information 
        record_list = list() 
        #Starts a counter that adds 1 everytime a fitting is performed
        model_counter = model_counter + 1
        #Starts a timer to end after the round of fitting is complete
        start_time = datetime.now()
        
        #Fits our model/batch_size and epochs are declared previously
        #model.fit(x = X_train, y = y_train.values, 
        #          batch_size = batch_size, epochs = epochs)
        
        history = model.fit(x = X_es_train, y = y_es_train.values, 
                  validation_data = (X_es_true, y_es_true),
                  batch_size = batch_size, epochs = epochs, 
                  callbacks = [es, mc])
        
        #Saves our model predictions
        y_pred = model.predict(X_true)
        
        #Saves our model run #, time to run, and model metrics to our temporary list
        record_list.extend([len(model_record)+1, #A model iteration count
                            loss, #Records the loss metric used to fit the model
                            format(datetime.now() - start_time), #Calculates the time it took to complete the model
                            len(history.history['loss']), #The epoch number of our model checkpoint 
                            r2_score(y_true, y_pred), #R2
                            mean_absolute_error(y_true, y_pred), #MAE
                            mean_squared_error(y_true, y_pred), #MSE
                            np.sqrt(mean_squared_error(y_true, y_pred)), #RMSE
                            max_error(y_true, y_pred) #MaxError
                           ])        
        
        #Adds the temporary list of model metrics (etc) to the end of a dataframe
        model_record.loc[len(model_record)] = record_list
        
        #Converts our predictions to a dataframe so it will play nice
        y_pred_df = pd.DataFrame(y_pred)
        
        #Adds predictions as a column to the end of a dataframe and names is accordingly
        predict_record = pd.concat([predict_record, y_pred_df], axis = 1)
        predict_record = predict_record.rename(columns = {0 : 'm' + str(model_counter[0])})
        
        #Calculates the residual values for each prediction and stores it as a dataframe 
        residuals_df = pd.DataFrame(abs(predict_record.iloc[:,len(predict_record.columns)-1] - predict_record.iloc[:,0]))
        
        #Adds y residuals as a column to the end of a dataframe and names is accordingly
        predict_record = pd.concat([predict_record, residuals_df], axis = 1)
        predict_record = predict_record.rename(columns = {0 : 'res' + str(model_counter[0])})

Epoch 1/250
1006/1006 [==============================] - 4s 3ms/step - loss: 0.2442 - mse: 0.2442 - val_loss: 0.2434 - val_mse: 0.2434
Epoch 2/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2394 - mse: 0.2394 - val_loss: 0.2431 - val_mse: 0.2431
Epoch 3/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2356 - mse: 0.2356 - val_loss: 0.2443 - val_mse: 0.2443
Epoch 4/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2342 - mse: 0.2342 - val_loss: 0.2390 - val_mse: 0.2390
Epoch 5/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2332 - mse: 0.2332 - val_loss: 0.2459 - val_mse: 0.2459
Epoch 6/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2310 - mse: 0.2310 - val_loss: 0.2435 - val_mse: 0.2435
Epoch 7/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2287 - mse: 0.2287 - val_loss: 0.2372 - val_mse: 0.2372
Epoch 8/250
1006/1006 [==============================] 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2093 - mse: 0.2093 - val_loss: 0.2178 - val_mse: 0.2178
Epoch 62/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2095 - mse: 0.2095 - val_loss: 0.2176 - val_mse: 0.2176
Epoch 63/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2096 - mse: 0.2096 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 64/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2097 - mse: 0.2097 - val_loss: 0.2189 - val_mse: 0.2189
Epoch 65/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2091 - mse: 0.2091 - val_loss: 0.2181 - val_mse: 0.2181
Epoch 66/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2093 - mse: 0.2093 - val_loss: 0.2212 - val_mse: 0.2212
Epoch 67/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2098 - mse: 0.2098 - val_loss: 0.2182 - val_mse: 0.2182
Epoch 68/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.2186 - val_mse: 0.2186
Epoch 122/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2087 - mse: 0.2087 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 123/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.2174 - val_mse: 0.2174
Epoch 124/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.2181 - val_mse: 0.2181
Epoch 125/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2087 - mse: 0.2087 - val_loss: 0.2188 - val_mse: 0.2188
Epoch 126/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2092 - mse: 0.2092 - val_loss: 0.2200 - val_mse: 0.2200
Epoch 127/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2211 - val_mse: 0.2211
Epoch 128/250
1006/1006 [==============================

Epoch 181/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 182/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2082 - mse: 0.2082 - val_loss: 0.2193 - val_mse: 0.2193
Epoch 183/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2187 - val_mse: 0.2187
Epoch 184/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2090 - mse: 0.2090 - val_loss: 0.2178 - val_mse: 0.2178
Epoch 185/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2194 - val_mse: 0.2194
Epoch 186/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 187/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2082 - mse: 0.2082 - val_loss: 0.2183 - val_mse: 0.2183
Epoch 188/250
1006/1006 [================

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 43/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2086 - mse: 0.2086 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 44/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2175 - val_mse: 0.2175
Epoch 1/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.2174 - val_mse: 0.2174
Epoch 2/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2194 - val_mse: 0.2194
Epoch 3/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 4/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.2197 - val_mse: 0.2197
Epoch 5/250
1006/1006 [==============================] - 3s 3ms/s

...

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 8/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 9/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 10/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 11/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 12/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 13/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 14/250
1006/1006 [==============================] - 3s 3m

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 14/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 15/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2175 - val_mse: 0.2175
Epoch 16/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 17/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2192 - val_mse: 0.2192
Epoch 18/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 19/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 20/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 74/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 75/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 76/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 77/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 78/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 79/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 80/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 9/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 10/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 11/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 12/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 13/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 14/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2166 - val_mse: 0.2166
Epoch 15/250
1006/1006 [==============================] - 3s 3

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 69/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 70/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 71/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 72/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 73/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 74/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 75/250
1006/1006 [==============================] - 3s 

In [13]:
model_record

,model_num,loss_type,time,epochs,r2,mae,mse,rmse,max_error
0,1,mean_squared_error,0:10:09.985911,199,0.188181,0.229925,0.216990,0.465822,5.240752
1,2,mean_squared_error,0:02:14.107263,44,0.190278,0.229074,0.216430,0.465220,5.226845
2,3,mean_squared_error,0:02:13.209607,44,0.187298,0.229439,0.217226,0.466075,5.212561
3,4,mean_squared_error,0:03:57.160640,78,0.182262,0.234028,0.218572,0.467517,5.219853
4,5,mean_squared_error,0:04:13.004387,84,0.190698,0.229194,0.216317,0.465099,5.233015
...,...,...,...,...,...,...,...,...,...
245,246,mean_squared_error,0:02:09.883465,42,0.193777,0.228176,0.215494,0.464214,5.228463
246,247,mean_squared_error,0:02:45.829959,54,0.191584,0.230442,0.216081,0.464845,5.227002
247,248,mean_squared_error,0:04:12.559628,81,0.191750,0.228678,0.216036,0.464797,5.212762
248,249,mean_squared_error,0:02:18.162961,44,0.192366,0.229215,0.215872,0.464620,5.211626


In [14]:
predict_record

,Acceleration3,m1,res1,m2,res2,m3,res3,m4,res4,m5,...,m246,res246,m247,res247,m248,res248,m249,res249,m250,res250
0,-0.314890,-0.126270,0.188620,-0.111108,0.203782,-0.096188,0.218702,-0.103069,0.211821,-0.113130,...,-0.101342,0.213548,-0.099672,0.215218,-0.085391,0.229499,-0.083165,0.231725,-0.087479,0.227411
1,-0.267150,-0.332794,0.065644,-0.275624,0.008474,-0.263741,0.003409,-0.253033,0.014117,-0.271467,...,-0.300220,0.033070,-0.285894,0.018744,-0.260028,0.007122,-0.248168,0.018982,-0.266242,0.000908
2,0.257710,0.248846,0.008864,0.265716,0.008006,0.265743,0.008033,0.373828,0.116118,0.304577,...,0.275419,0.017709,0.330853,0.073143,0.259564,0.001854,0.301639,0.043929,0.269031,0.011321
3,-0.193330,-0.118344,0.074986,-0.104031,0.089299,-0.089541,0.103789,-0.096700,0.096630,-0.108859,...,-0.101965,0.091365,-0.100436,0.092894,-0.086183,0.107147,-0.084694,0.108636,-0.088205,0.105125
4,-0.409560,-0.134593,0.274967,-0.118540,0.291020,-0.103169,0.306391,-0.110299,0.299261,-0.124500,...,-0.128355,0.281205,-0.123984,0.285576,-0.107657,0.301903,-0.104337,0.305223,-0.111139,0.298421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,-0.021237,-0.100872,0.079635,-0.088430,0.067193,-0.074887,0.053650,-0.082659,0.061422,-0.095581,...,-0.086500,0.065263,-0.085790,0.064553,-0.087928,0.066691,-0.088066,0.066829,-0.089805,0.068568
21447,-0.264320,0.073848,0.338168,0.079087,0.343407,0.096215,0.360535,0.065287,0.329607,0.087902,...,0.095853,0.360173,0.082392,0.346712,0.101734,0.366054,0.096200,0.360520,0.091499,0.355819
21448,0.060178,-0.127544,0.187722,-0.112247,0.172425,-0.097257,0.157435,-0.104094,0.164272,-0.113817,...,-0.101242,0.161420,-0.099549,0.159727,-0.085264,0.145442,-0.082919,0.143097,-0.087362,0.147540
21449,0.535320,0.223804,0.311516,0.242471,0.292849,0.243053,0.292267,0.291043,0.244277,0.266138,...,0.233925,0.301395,0.287933,0.247387,0.218114,0.317206,0.257256,0.278064,0.228730,0.306590


In [15]:
#Nifty call to confirm our variables were properly inputted into our model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 4,353
Trainable params: 4,353
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_record

,model_num,loss_type,time,epochs,r2,mae,mse,rmse,max_error
0,1,mean_squared_error,0:10:09.985911,199,0.188181,0.229925,0.216990,0.465822,5.240752
1,2,mean_squared_error,0:02:14.107263,44,0.190278,0.229074,0.216430,0.465220,5.226845
2,3,mean_squared_error,0:02:13.209607,44,0.187298,0.229439,0.217226,0.466075,5.212561
3,4,mean_squared_error,0:03:57.160640,78,0.182262,0.234028,0.218572,0.467517,5.219853
4,5,mean_squared_error,0:04:13.004387,84,0.190698,0.229194,0.216317,0.465099,5.233015
...,...,...,...,...,...,...,...,...,...
245,246,mean_squared_error,0:02:09.883465,42,0.193777,0.228176,0.215494,0.464214,5.228463
246,247,mean_squared_error,0:02:45.829959,54,0.191584,0.230442,0.216081,0.464845,5.227002
247,248,mean_squared_error,0:04:12.559628,81,0.191750,0.228678,0.216036,0.464797,5.212762
248,249,mean_squared_error,0:02:18.162961,44,0.192366,0.229215,0.215872,0.464620,5.211626


In [17]:
predict_record

,Acceleration3,m1,res1,m2,res2,m3,res3,m4,res4,m5,...,m246,res246,m247,res247,m248,res248,m249,res249,m250,res250
0,-0.314890,-0.126270,0.188620,-0.111108,0.203782,-0.096188,0.218702,-0.103069,0.211821,-0.113130,...,-0.101342,0.213548,-0.099672,0.215218,-0.085391,0.229499,-0.083165,0.231725,-0.087479,0.227411
1,-0.267150,-0.332794,0.065644,-0.275624,0.008474,-0.263741,0.003409,-0.253033,0.014117,-0.271467,...,-0.300220,0.033070,-0.285894,0.018744,-0.260028,0.007122,-0.248168,0.018982,-0.266242,0.000908
2,0.257710,0.248846,0.008864,0.265716,0.008006,0.265743,0.008033,0.373828,0.116118,0.304577,...,0.275419,0.017709,0.330853,0.073143,0.259564,0.001854,0.301639,0.043929,0.269031,0.011321
3,-0.193330,-0.118344,0.074986,-0.104031,0.089299,-0.089541,0.103789,-0.096700,0.096630,-0.108859,...,-0.101965,0.091365,-0.100436,0.092894,-0.086183,0.107147,-0.084694,0.108636,-0.088205,0.105125
4,-0.409560,-0.134593,0.274967,-0.118540,0.291020,-0.103169,0.306391,-0.110299,0.299261,-0.124500,...,-0.128355,0.281205,-0.123984,0.285576,-0.107657,0.301903,-0.104337,0.305223,-0.111139,0.298421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,-0.021237,-0.100872,0.079635,-0.088430,0.067193,-0.074887,0.053650,-0.082659,0.061422,-0.095581,...,-0.086500,0.065263,-0.085790,0.064553,-0.087928,0.066691,-0.088066,0.066829,-0.089805,0.068568
21447,-0.264320,0.073848,0.338168,0.079087,0.343407,0.096215,0.360535,0.065287,0.329607,0.087902,...,0.095853,0.360173,0.082392,0.346712,0.101734,0.366054,0.096200,0.360520,0.091499,0.355819
21448,0.060178,-0.127544,0.187722,-0.112247,0.172425,-0.097257,0.157435,-0.104094,0.164272,-0.113817,...,-0.101242,0.161420,-0.099549,0.159727,-0.085264,0.145442,-0.082919,0.143097,-0.087362,0.147540
21449,0.535320,0.223804,0.311516,0.242471,0.292849,0.243053,0.292267,0.291043,0.244277,0.266138,...,0.233925,0.301395,0.287933,0.247387,0.218114,0.317206,0.257256,0.278064,0.228730,0.306590


In [18]:
model_record.to_csv('data/rnn_results_1_2.csv')
predict_record.to_csv('data/rnn_predictions_1_2.csv')